### This notebook is based of: https://www.kaggle.com/abnera/transfer-learning-keras-xception-cnn

In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.layers import Input, Flatten, Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.applications.xception import Xception, preprocess_input, decode_predictions

Using TensorFlow backend.


In [2]:
def create_generators(train_data_dir, validation_data_dir):
    # Read Data and Augment it: Make sure to select augmentations that are appropriate to your images.

    transformation_ratio = .05  # how aggressive will be the data augmentation/transformation

    train_datagen = ImageDataGenerator(rescale=1. / 255,
                                       rotation_range=transformation_ratio,
                                       shear_range=transformation_ratio,
                                       zoom_range=transformation_ratio,
                                       cval=transformation_ratio,
                                       horizontal_flip=True,
                                       vertical_flip=True)

    train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                        target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                        batch_size=BATCH_SIZE,
                                                        class_mode='categorical')

    validation_datagen = ImageDataGenerator(rescale=1. / 255)
    
    validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                                  target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                                  batch_size=BATCH_SIZE,
                                                                  class_mode='categorical')
    return train_generator, validation_generator


def create_model(num_classes):
        base_model = Xception(input_shape=(IMG_WIDTH, IMG_HEIGHT, 3), weights='imagenet', include_top=False)

        # Top Model Block
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        predictions = Dense(num_classes, activation='softmax')(x)

        # add your top layer block to your base model
        model = Model(base_model.input, predictions)
        
        for layer in model.layers[:-10]:
            layer.trainable = False
        
        model.compile(optimizer='nadam',loss='categorical_crossentropy', metrics=['accuracy'])
        return model


def train(train_generator, validation_generator, model):
    model.fit_generator(train_generator,
                        epochs=1,
                        validation_data=validation_generator,
                        steps_per_epoch=3,
                        validation_steps=2,
                        verbose=1)

# makes the prediction of the file path image passed as parameter 
def predict(file, model, to_class):
    im = load_img(file, target_size=(IMG_WIDTH, IMG_HEIGHT))
    x = img_to_array(im)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    index = model.predict(x).argmax()
    return to_class[index]

In [13]:
DIR = "./"
IMG_WIDTH, IMG_HEIGHT = 299, 299 # set this according to keras documentation, each model has its own size
BATCH_SIZE = 200 # decrease this if your computer explodes

In [14]:
train_generator, validation_generator = create_generators(DIR + "labelled", DIR + "validation")

total_classes = len(train_generator.class_indices) # necesary to build the last softmax layer
to_class = {v:k for k,v in train_generator.class_indices.items()} # usefull when model returns prediction
to_class

Found 1953 images belonging to 7 classes.
Found 397 images belonging to 7 classes.


{0: 'bathroom',
 1: 'bedroom',
 2: 'exterior',
 3: 'kitchen',
 4: 'livingroom',
 5: 'plane',
 6: 'publicity'}

In [5]:
m = create_model(total_classes)

In [6]:
# Run this several times until you get good acurracy in validation (wachout of overfitting)
train(train_generator, validation_generator, m)

Epoch 1/1
3/3 [==============================] - 478s - loss: 1.1786 - acc: 0.5667 - val_loss: 0.4704 - val_acc: 0.8463


In [7]:
train(train_generator, validation_generator, m)

Epoch 1/1
3/3 [==============================] - 490s - loss: 0.4296 - acc: 0.8600 - val_loss: 0.4184 - val_acc: 0.8589


In [8]:
train(train_generator, validation_generator, m)

Epoch 1/1
3/3 [==============================] - 399s - loss: 0.4583 - acc: 0.9064 - val_loss: 0.3271 - val_acc: 0.8841


In [9]:
train(train_generator, validation_generator, m)

Epoch 1/1
3/3 [==============================] - 496s - loss: 0.2934 - acc: 0.9133 - val_loss: 0.3055 - val_acc: 0.8967


In [10]:
train(train_generator, validation_generator, m)

Epoch 1/1
3/3 [==============================] - 414s - loss: 0.6041 - acc: 0.7714 - val_loss: 0.5842 - val_acc: 0.8539


In [11]:
train(train_generator, validation_generator, m)

Epoch 1/1
3/3 [==============================] - 517s - loss: 0.2822 - acc: 0.9033 - val_loss: 0.3129 - val_acc: 0.9118


In [12]:
train(train_generator, validation_generator, m)

Epoch 1/1
3/3 [==============================] - 432s - loss: 0.4567 - acc: 0.8089 - val_loss: 0.4450 - val_acc: 0.8640


In [13]:
train(train_generator, validation_generator, m)

Epoch 1/1
3/3 [==============================] - 501s - loss: 0.2308 - acc: 0.9367 - val_loss: 0.2950 - val_acc: 0.9043


In [14]:
train(train_generator, validation_generator, m)

Epoch 1/1
3/3 [==============================] - 526s - loss: 0.1817 - acc: 0.9450 - val_loss: 0.2853 - val_acc: 0.9118


In [ ]:
# execute this when you want to save the model
# m.save('wiiii-finished')

# execute this when you want to load the model
# from keras.models import load_model
# m_i_am_back = load_model('wiiii-finished')